# N-Body

### N-body problem

The N-body problem is the problem concerning the prediction of the motion of a group of objects interacting gravitationally with eachother. This puzzle is of paramount importance in astrophysics because is applied to a huge variety of astrophysical situations and scales (solar system, satellites, binary evolution, stars in a cluster or in a galaxy...). \
The N-body problem dates back to the XVII century when Isaac Newton came up with the intuition that two bodies with mass $m_1$ and $m_2$ at a distance $\vec{r}_{12}$ from eachother are subjected to a mutual force
$$
\begin{equation}
    \vec{F}_{1,2} = m_1 \vec{a}_1 = - G \frac{m_1 m_2}{r_{12}^2} \frac{ \vec{r}_{12} }{ r_{12} } .
\end{equation}\tag{1}
$$
Then, extending this to a system of N-body, the acceleration of the body $i$ due to all the other bodies is expressed by 
$$
\begin{equation}
    \frac{ d^2 \vec{x}_i } {dt^2} = - G \sum^N_{j=1, j\neq i} m_j \frac{ \vec{x}_i - \vec{x}_j }{ | \vec{x}_i - \vec{x}_j |^3  } .
\end{equation}\tag{2}
$$

It can be demonstrated that each N-body system has:
- 6 costants of motion (the position and the velocity of the centre of mass);
- 4 integrals of motion (the energy and the three components of the angular momentum).

It exists an analitic solution for $N=2$ (Bernoulli was the first to derive it in 1710). The complete solution for a generic number of bodies had not been found yet (in 1991 Qiudong Wang found a convergent power series solution for a generic number of bodies, but it is too difficult to implement and has a slow convergence). In fact, by simply adding a third particle the unknowns become 18, and the constants/integrals of motion are not enough to reduce the complexity of the problem. Then constrains have to be added in order to achieve an analitic solution, and this analitic solution is known only for a small amount of cases (e.g. circular restricted three body problem). Therefore, numerical methods are used in order to integrate this system of differential equations. 

Another important thing to point out is the numerical complexity of this problem, which is nothing more than the number of computations (numerical iterations) required to complete the task. In the case examined the time complexity is $O(N^2)$: it grows rapidly as N increases, and this represents a major problem to overcome when building a model to describe the system. 

### N-body units

N-body units are a very powerful and useful method when dealing with the N-body problem. They represent a convenient set of units for N-body simulations, based on the assumption that $G = 1$. \
The conversion to physical values can be done a-posteriori, inserting eventually some typical values of the astrophysical system of interest. \
It must be kept in mind that this treatment fails when adding to the simulation data about stars' proper mass and radius, or stellar evolution, or SN explosion, or when setting additional fields in order to reproduce a certan galactic physical configuration; on the other hand, when stars are considered as abstract point mass particles, the scale invariance of the N-body simulation can be exploited.

In order to convert N-body units into physical units a scale lenght $L_{scale}$ and a scale mass $M_{scale}$ need to be defined. Then:
$$
\begin{equation}
    T_{scale} = \sqrt{  \frac{L_{scale}^3}{G M_{scale}}   }   ,
\end{equation}\tag{3}
$$
$$
\begin{equation}
    V_{scale} = \frac{L_{scale}}{T_{scale}} = \sqrt{  \frac{G M_{scale}}{L_{scale}}   } ,
\end{equation}\tag{4}
$$
and from here physical units can be recovered
$$
\begin{gather}
    L_{phys}= L_{Nbody} L_{scale}  ,     \\

    M_{phys}= M_{Nbody} M_{scale}  ,     \\

    T_{phys}= T_{Nbody} T_{scale}  ,     \\

    V_{phys}= V_{Nbody} V_{scale} .
\end{gather}\tag{5}
$$


Anyway, physical units can be easily recovered with the Fireworks package, in particular with the aid of the subpackage `nunits`.

# Collisional vs Collisionless Simulations

The importance of close particle-particle interactions allows to distinguish between collisional and collisionless systems. To do so, the starting point would be an analysis of if and how much gravitational encounters between stars are able to change the kinematic status of themselves. \
Consider a test star with velocity $v$ which crosses the potential well of another star a rest. Both stars have the same mass $m$, and $b$ is their mutual distance. After some calculation it can be seen that the velocity change of the incoming star is $\delta v = \frac{2 G m}{b v}$. \
Extending now this reasoning considering all the stars in the galaxy, it is easy to prove that the average velocity change $\delta v$ is zero, but the standard deviation $\delta v^2$ is not. In particular one can find out that , being $N$ the number of stars in the system,
$$
\begin{equation}
    \frac{ \Delta v^2 }{v^2} \approx 8 \frac{1}{N} log\frac{N}{2} .
\end{equation}\tag{6}
$$
This implies that to produce a substantial change in the velocity the star needs to cross the galaxy a number of times $n_{relax}$ equal to 
$$
\begin{equation}
    n_{relax} \approx  \frac{N}{ 8 log \frac{N}{2} } ;
\end{equation}\tag{7}
$$
from here, defining $t_{cross} \approx \frac{R}{v}$ as the time required approximately from the star to cross the whole cluster, the concept of relaxation time can be introduced
$$
\begin{equation}
    t_{relax} = n_{relax} t_{cross} ,
\end{equation}\tag{8}
$$
and substituting in this equation the previous expression
$$
\begin{equation}
    t_{relax} \approx 0.1 \frac{N}{log N } t_{cross} .
\end{equation}\tag{9}
$$
Overall, these times can be used to distinghish between collisional and collisionless systems. In particular:

1) $t_{simulation} \gtrsim t_{relax}$: _COLLISIONAL SYSTEMS_ \
In this case particle-particle interactions cannot be neglected in order to describe particles' motion: direct force extimate has to be used.
2) $t_{simulation} \ll t_{relax}$: _COLLISIONLESS SYSTEMS_ \
In this case particle-particle interactions do not have an important role, particles' trajectories can be tracked considering a smooth matter distribution: less computationally expensive tecniques are required to estimate the force.

Which category does the system in question belong to?

In [1]:
import numpy as np
import pandas as pd
from fireworks.particles import Particles

In [2]:
df = pd.read_csv("data_cvs/Nbody_disc.csv")         # data of the system
N = df.shape[0]                                     # number of objects of our problem

# creating a class with the mass, pos, vel of each element of the system
masses = df[['mass']]
positions = df[['x', 'y', 'z']]
velocities = df[['vx', 'vy', 'vz']]
masses = masses.values.reshape(-1)                                      # reshape mass array to 1D array (required by pyfalcon)
sys = Particles(positions.values, velocities.values, masses)            # using the class particles to define our system and use some useful tools

# applying our formulas written above
t_cross = np.nanmin(sys.radius()/sys.vel_mod())         # minimum crossing time; the minimum is used so to have the maximum crossing time and consequently the maximum relaxation time
t_simulation = 210                                      # given
n_relax = 0.1 * N/np.log(N)                             # n_relax
t_relax = n_relax * t_cross                             # t_relax for the minimum crossing time

# taking the ratio to compare
ratio = t_simulation/t_relax

print(ratio)


0.02161794525105976


/tmp/ipykernel_84962/769051311.py:12: RuntimeWarning: invalid value encountered in divide
  t_cross = np.nanmin(sys.radius()/sys.vel_mod())         # minimum crossing time; the minimum is used so to have the maximum crossing time and consequently the maximum relaxation time


Therefore, $\min \left( \frac{t_{simulation}}{t_{relax}} \right) = 0.0216 \ll 1 \implies t_{simulation} \ll t_{relax} \implies$ COLLISIONLESS SYSTEM. \
Consequently it is possible to reduce the complexity of the simulation treating the system as a fluid moving in phase space and neglecting collisions (and formation of binaries as well).

# Force Estimate in Collisionless Simulations Based on Tree Algorithms

As moving toward the description of collisionless systems, one of the things that will come in handly are the tree codes, in particular the Barnes & Hut algoritm and the Dehen algoritm. These approaches are based on the approximation of long range forces so to make force estimate easier and faster.  
A brief discussion of these two is presented in the following.

### Barnes & Hut Algorithm

This procedure consists in grouping, clustering together distant particles so to create a super-particle. This super-particle is then considered as a single particle and assimilated to its centre of mass: it has $M = \sum m_i$, $\vec{r} = \frac{ \sum {m_i \vec{r}_i} } { M } $, $\vec{v} = \frac{ \sum {m_i \vec{v}_i } } { M } $. In this way it is possible to compute the force of all the particles in the cluster in a single step, greatly speeding up the whole process.  The crucial part of this algorithm is the clustering procedure adopted.  

Barnes & Hut (1986) proposed to use an oct-tree: the 3D volume considered in the simulation is recursively divided into cubic cells until each cell hosts one particle at most. In this way only particles from nearby cells need to be treated individually, whereas particles from distant cells can be assimilated with the super-particle described before. In this way the complexity of the problem is reduced from $O(N^2)$ to $O(N logN)$.\
In order to discriminate nearby from far-away particles the concept of opening angle can be introduced. In fact, calling $S_{branch}$ the dimension of the considered tree branch and $D_{i, branch}$ the distance between the particle considered and the centre of mass of the branch, opening angle is defined as
$$
\begin{equation}
    \theta = \frac{ S_{branch} } { D_{i, branch} } .
\end{equation}\tag{10}
$$
Two scenarios emerge from here:
1) if $\theta < \theta_{crit}$: the branch is distant enough, and the force can be calculated;
2) if $\theta > \theta_{crit}$: the branch is too close, and its sub-branches sould be studied, iterating this reasoning.

$\theta_{crit}$ is arbitraty chosen; a commonly used value is 0.5.

### Dehen Algorithm

Generally, a potential can be written as a  series of successive terms, called multipole expansion. In particular
$$
\begin{equation}
    \Phi_B( \vec{r} ) = \sum \Phi( \vec{r} - \vec{r_i} ) = 
    \frac{ M }{| \vec{r} | }      +   
    \frac{ D }{| \vec{r} |^3 }    +  
    \frac{ Q }{| \vec{r} |^5 }    + ... ,
\end{equation}\tag{11}
$$
where the first term indicates the Monopole component, the second term the Dipole component and the third term the Quadrupole component. \
The multiple moments of each cube can be determined iteratively during the construction of the tree. \
Now, while the Barnes & Hut algoritm stops at the monopole whithout considering successive terms, the Dehen algoritm exploits the multipole approximation and other things (for example mutual interaction between branches) to develop a faster and more efficient tree code. 

This algorithm is used in the Fireworks package, in particular in the `acceleration_pyfalcon` extimate for the acceleration.

# Short Description of the Fireworks Package and its Implementation

### Fireworks

Fireworks is the python package used to run the simulations. It is a library that contains tools to initialize and evolve N-body systems, and it can be used to simulate collisionless systems, collisional systems and orbit integration. The basic skeleton of the function is already provided, so to make the implementation straightforward. It also contains some other useful tool as `pyfalcon` and `TSUNAMI`. \
The raw version of Fireworks contains the following submodules: 
- `particles`: it contains the class `Particles` in which all the information about the particles can be stored (position, velocity, mass), getting some useful tool for the simulations (e.g.radius, velocity module);
- `ic`: it contains some function through which initial conditions can be generated. The subpackage contains two functions: `ic_random_normal`, which draws initial contitions from a normal distrubution, and `ic_two_body`, which generates initial conditions for the case of a two-body system;
- `Version`: it contains the variable `_version_` in the format MAJOR.MINOR:PATH ;
- `Nbodylib`: it contains functions so to estimate the gravitational forces and acceleration. It is in turn subdivided into other four cathegories:
    - `dynamics`: it contains functions to estimate accelerations due to gravitational forces. `acceleration_pyfalcon` (discussed above when introducing the Dehen algorithm) is present;
    - `integrators`: it contains integrators used to integrate the ODE equations of the motion and evolve the system in time. The `TSUNAMI` integrator is in there;
    - `nunits`: it contains the class `Nbody_units`, necessary to transform the data from physical units to nbody units or viceversa;
    - `potentials`: it contains functions to estimate the acceleration due to a smooth potential
    - `timesteps`: it contains functions to estimate the timestep for the N-body integrations. This naturally depends on the current properties of the system. `adaptive_timestep_simple`, which uses the formula $t_s = \min \left( \frac{r}{v} \right)$ is present.

### Implementation

To run the simulation it was necessary to implement the code. Here a brief summary of this implementation is presented. As done before, the modules are listed one by one, and the main changes are highlighted.
- `particles`: a method to compute the potential and the kinetic energy was added;
- `ic`: a funcion `ic_random_uniform` was written using the template of the previous ones, to generate initial conditions from a uniform distribution. 
- `Version`: no implementations for this subpackage;
- `Nbodylib`: this is where the most of the work was focused (obviously). Going into details:
    - `dynamics`: using the template, three functions were created: `acceleration_direct`, which uses two for cycles, `acceleration vectorized`, which uses vectorial operations (faster), and `acceleration_jerk_direct`, which computes the acceleration with pyfalcon and the jerk with two for cycles;
    - `integrators`: the template was used to build the `Euler` integrator, the `Leapfrog` integrator and the `Runge-Kuta` integrator;
    - `nunits`: no implementations for this subpackage;
    - `potentials`: <div class='alert alert-danger' role='alert'> completa </div>;
    - `timesteps`: a new function `adaptive_timestep_r` was implemented, which adds a factor $\eta=0.1$ to the previous formula. This value was chosen  so to have a good balance between resolution and computational time.
